In [ ]:
import numpy as np
from typing import Tuple, Dict
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))
import pandas as pd
import tensorflow as tf
from tensorflow.math import logical_not
from builderfuncs import EarlyStopAndSave
from builderfuncs import build_gru, build_bigru, build_bilstm, build_lstm, save_whole_model, restore_model
from parameterdicts import GRUParameters
from ktfuncs import *
import keras_tuner as kt
import os
import datetime


df = pd.read_csv('data/scaled_U2_data.csv', index_col=0)
df.drop("UNNAMED: 0", axis=1, inplace=True)

devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

print("Num GPUs Available: ", len(devices))

In [ ]:
def ttv_split(ds: tf.data.Dataset, ds_size: int, 
              train_split: float = 0.8, 
              val_split: float = 0.1, 
              test_split: float = 0.1) -> Tuple[tf.data.Dataset, 
                                                tf.data.Dataset, 
                                                tf.data.Dataset]:

    assert (train_split + test_split + val_split) == 1
   
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds


def split_window(features: tf.Tensor) -> Tuple[Dict[tf.Tensor, tf.Tensor],
                                               tf.Tensor]:
    inputs = features[:12, 1:]
    state_labels = features[12, 75:]
    targ_labels = tf.expand_dims(features[11, 0], axis=0)
    labels = tf.concat([targ_labels, state_labels], axis=0)
    inputs.set_shape([12, 247])
    labels.set_shape([174])
    

    return inputs, labels

                        
def make_dataset(data: pd.DataFrame, length: int, 
                 batch_size: int = 64, multistep: bool = True) -> Tuple[tf.data.Dataset, 
                                                                        tf.data.Dataset, 
                                                                        tf.data.Dataset]:
    data = np.array(data.iloc[:, :], dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(data=data,
                                                        targets=None,
                                                        sequence_length=length,
                                                        sequence_stride=1,
                                                        shuffle=True,
                                                        seed=1,
                                                        batch_size=None)

    ds = ds.filter(lambda x: tf.reduce_all(logical_not(tf.math.is_nan(x))))
    ds = ds.map(split_window).batch(batch_size)
    ds = ds.apply(tf.data.experimental.assert_cardinality(170680//batch_size + 1))
    train_ds, val_ds, test_ds = ttv_split(ds, 170680//batch_size, train_split=0.8, val_split=0.1, test_split=0.1)
    train_ds = train_ds
    return train_ds, val_ds, test_ds

In [ ]:
gru_logdir = os.path.join("logs", f'gru{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
gru_tensorboard_callback = tf.keras.callbacks.TensorBoard(gru_logdir, histogram_freq=1)
gru_earlystopper = EarlyStopAndSave(filepath="model_folder/gru_folder", patience=3, lim=0.005, minormax="max")

bigru_logdir = os.path.join("logs", f'bigru{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
bigru_tensorboard_callback = tf.keras.callbacks.TensorBoard(bigru_logdir, histogram_freq=1)
bigru_earlystopper = EarlyStopAndSave(filepath="model_folder/bigru_folder", patience=3, lim=0.005, minormax="max")

lstm_logdir = os.path.join("logs", f'lstm{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
lstm_tensorboard_callback = tf.keras.callbacks.TensorBoard(lstm_logdir, histogram_freq=1)
lstm_earlystopper = EarlyStopAndSave(filepath="model_folder/lstm_folder", patience=3, lim=0.005, minormax="max")

bilstm_logdir = os.path.join("logs", f'bilstm{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
bilstm_tensorboard_callback = tf.keras.callbacks.TensorBoard(bilstm_logdir, histogram_freq=1)
bilstm_earlystopper = EarlyStopAndSave(filepath="model_folder/bilstm_folder", patience=3, lim=0.005, minormax="max")

train_data, val_data, test_data = make_dataset(df, 23, batch_size=64)

In [ ]:
gru_tuner = kt.BayesianOptimization(kt_gru,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="gru_trial_1",
                                    seed=1)
bigru_tuner = kt.BayesianOptimization(kt_bigru,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="bigru_trial_1",
                                    seed=1)
lstm_tuner = kt.BayesianOptimization(kt_lstm,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="lstm_trial_1",
                                    seed=1)
bilstm_tuner = kt.BayesianOptimization(kt_bilstm,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="bilstm_trial_1",
                                    seed=1)



In [ ]:
parameters: GRUParameters = gru_tuner.get_best_hyperparameters()[0].values
gru = build_gru(parameters=parameters, name="gru_model", dynamic=False)

parameters: GRUParameters = bigru_tuner.get_best_hyperparameters()[0].values
bigru = build_bigru(parameters=parameters, name="bigru_model", dynamic=False)

parameters: GRUParameters = lstm_tuner.get_best_hyperparameters()[0].values
lstm = build_lstm(parameters=parameters, name="lstm_model", dynamic=False)

parameters: GRUParameters = bilstm_tuner.get_best_hyperparameters()[0].values
bilstm = build_bilstm(parameters=parameters, name="bilstm_model", dynamic=False)

In [ ]:
gru.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[gru_earlystopper, gru_tensorboard_callback])
bigru.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[bigru_earlystopper, bigru_tensorboard_callback])
lstm.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[lstm_earlystopper, lstm_tensorboard_callback])
bilstm.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[bilstm_earlystopper, bilstm_tensorboard_callback])

In [ ]:
%tensorboard --gru_logdir logs

In [ ]:
%tensorboard --bigru_logdir logs

In [ ]:
%tensorboard --lstm_logdir logs

In [ ]:
%tensorboard --bilstm_logdir logs